In [10]:
!pip install selenium

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import time
import random

# User's profile page
user_profile = 'Jebby_Lin'


login = False
username = 'username'
password = 'password'

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') -- using headless mode currently leads to NoSuchElementException 
driver = webdriver.Chrome('./chromedriver', options=options)

# get a reference to the download menu. This will run before the page has 
# finished loading, so we stick it in a while loop and just keep looping
# until we're successful.
def find_by_id(element_id):
    while True:
        try:
            target = driver.find_element_by_id(element_id)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target

def find_by_class(element_class):
    while True:
        try:
            target = driver.find_element_by_class_name(element_class)
        except NoSuchElementException:
            time.sleep(0.2)
            continue
        else:
            break
    return target


if login:
    driver.get("https://myanimelist.net/login.php?from=%2F")
    driver.implicitly_wait(5)
    driver.find_element_by_xpath("//*[@id='loginUserName']").send_keys(username)
    driver.find_element_by_xpath("//*[@id='login-password']").send_keys(password)
    driver.find_element_by_xpath("//*[@class='inputButton btn-form-submit btn-recaptcha-submit']").click()
    
    
user_link = "https://myanimelist.net/profile/" + user_profile
driver.get(user_link)
driver.get(user_link)
driver.implicitly_wait(3)



In [11]:
all_df = pd.DataFrame()

for i in [1, 2, 4]:
    status = i
    user_link = "https://myanimelist.net/animelist/" + user_profile + "?status=" + str(i) + "&tag="

    driver.get(user_link)

    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    mal_list = find_by_id('list-container')
    table = mal_list.find_element_by_tag_name('table')
    rows = table.find_elements_by_tag_name('tbody')
    #header_row = rows[0]
    rows_text = [row.text.split('\n') for row in rows[1:]]
    #df = pd.DataFrame(rows_text, columns = header_row.text.split('\n'))
    df = pd.DataFrame(rows_text, columns = [i for i in np.arange(1, 6)])
    
    df = df.drop([2], axis=1)
    df["Title"] = (df[1].str.split(None, 1).str[1])
    df = df.drop(1, axis=1)
    df["Score"] = (df[3].str.split(None, 1).str[0])
    df["Type"] = (df[3].str.split(None, 1).str[1])
    df = df.drop(3, axis=1)
    df["Rated"] = (df[5].str.split(None, 3).str[0])
    df["Aired Month"] = (df[5].str.split(None, 3).str[1])
    df["Aired Year"] = (df[5].str.split(None, 3).str[2])
    df["Studio"] = (df[5].str.split(None, 3).str[3])
    df = df.drop(5, axis=1)
    df = df.rename(columns={4: "Episodes"})
    df['Title'] = df['Title'].map(lambda x: x.replace(' Watch Episode Video', ''))
    
    if status != 2:
        ep_split = df["Episodes"].str.split(" / ")
        df["Episodes Watched"] = ep_split.str[0]
        df["Episodes"] = ep_split.str[1]
    else:
        df["Episodes Watched"] = df["Episodes"]
        
    
    if status == 1:
        df["Status"] = "Currently Watching"
    elif status == 2:
        df["Status"] = "Completed"
    elif status == 3:
        df["Status"] = "On-Hold"
    elif status == 4:
        df["Status"] = "Dropped"
    elif status == 6:
        df["Status"] = "Plan to Watch"
    else:
        print("Error in status #")
        
    
    all_df = pd.concat([all_df, df])

In [12]:
all_df = all_df.rename(columns={"Score":"User Score"})
all_df

,Episodes,Title,User Score,Type,Rated,Aired Month,Aired Year,Studio,Episodes Watched,Status
0,22,3-gatsu no Lion,9,TV,PG-13,Fall,2016,Shaft,22,Completed
1,22,3-gatsu no Lion 2nd Season,10,TV,PG-13,Fall,2017,Shaft,22,Completed
2,24,Accel World,7,TV,PG-13,Spring,2012,Sunrise,24,Completed
3,50,Ahiru no Sora,7,TV,PG-13,Fall,2019,Diomedéa,50,Completed
4,13,Ajin,6,TV,R,Winter,2016,Polygon Pictures,13,Completed
...,...,...,...,...,...,...,...,...,...,...
176,224,Yu☆Gi☆Oh! Duel Monsters,10,TV,PG-13,Spring,2000,Gallop,224,Completed
0,366,Bleach,2,TV,PG-13,Fall,2004,Studio Pierrot,63,Dropped
1,5,Dimension W: W no Tobira Online - Rose no Onay...,-,Special,G,Winter,2016,Studio 3Hz,1,Dropped
2,51,Metal Fight Beyblade,-,TV,PG,Spring,2009,"Tatsunoko Production, SynergySP, Seven",25,Dropped


In [13]:
all_df.to_csv(user_profile + ' Simple Ratings.csv')